<a href="https://colab.research.google.com/github/MFathimaSudheer17800/Smart-Campus-Navigation-/blob/main/Smart_Campus_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SMART CAMPUS NAVIGATION

In [ ]:
!pip install gTTS qrcode[pil] networkx matplotlib
import networkx as nx
import matplotlib.pyplot as plt
from gtts import gTTS
from IPython.display import display, Markdown, Audio, Image
import qrcode
import time

# ----------------- Speak Function -----------------
def speak_steps(lines):
    for i, line in enumerate(lines, start=1):
        filename = f"step_{i}.mp3"
        tts = gTTS(line)
        tts.save(filename)
        display(Audio(filename, autoplay=True))
        time.sleep(4)

# ----------------- Graph Setup -----------------
G = nx.DiGraph()
edges = [
    ("Entrance", "Office"),
    ("Office", "Principal Office"),
    ("Office", "Seminar Hall 2"),
    ("Office", "CC1"),
    ("CC1", "CC2"),
    ("CC2", "CC3"),
    ("CC3", "Seminar Hall 1"),
    ("Seminar Hall 1", "CSE Dept"),
    ("CSE Dept", "ECE Dept"),
    ("ECE Dept", "EEE Dept")
]
G.add_edges_from(edges)

# Room numbers
room_numbers = {
    "Seminar Hall 1": "307",
    "CSE Dept": "316",
    "ECE Dept": "318",
    "CC1": "201",
    "CC2": "201",
    "CC3": "205",
    "Seminar Hall 2": "208",
    "Office": "209A",
    "Principal Office": "212",
    "EEE Dept": "417"
}

# Floor mapping
floor_mapping = {
    "2": "2nd Floor",
    "3": "3rd Floor",
    "4": "4th Floor"
}

# ----------------- Map Drawing -----------------
def generate_directional_image(graph):
    plt.figure(figsize=(8, 6))
    pos = {
        "Entrance": (0, 0),
        "Office": (0, 1),
        "Principal Office": (1, 1),
        "Seminar Hall 2": (-1, 1),
        "CC1": (0, 2),
        "CC2": (1, 2),
        "CC3": (2, 2),
        "Seminar Hall 1": (0, 3),
        "CSE Dept": (1, 3),
        "ECE Dept": (2, 3),
        "EEE Dept": (1, 4)
    }
    labels = {node: f"{node}\n({room_numbers[node]})" if node in room_numbers else node
              for node in graph.nodes()}
    nx.draw(graph, pos, labels=labels, node_color="lightblue",
            node_size=2500, font_size=9, font_weight="bold")
    plt.title("College Navigation Map (Floor-wise)", fontsize=12, fontweight="bold")
    img_path = "college_map.png"
    plt.savefig(img_path)
    plt.close()
    return img_path

# ----------------- QR Code Generator for Drive Link -----------------
drive_link = "https://drive.google.com/file/d/1_tlssjsG9tvBTOVaf-XpR4I4DqUfsPiT/view?usp=drivesdk"

def generate_qr_for_drive_link(link, qr_filename="ROUTE MAP.png"):
    qr = qrcode.QRCode(
        version=6,
        error_correction=qrcode.constants.ERROR_CORRECT_H,
        box_size=10,
        border=4,
    )
    qr.add_data(link)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white")
    qr_img.save(qr_filename)
    display(Image(qr_filename))
    print("✅ Scan this QR to open the full route map on Google Drive.")
    return qr_filename

# ----------------- Route Finder -----------------
def find_route(start, end):
    lines = []

    # Floor info + destination
    if end in room_numbers:
        room = room_numbers[end]
        floor_digit = room[0]
        if floor_digit in floor_mapping:
            lines.append(f"From Entrance, go to {floor_mapping[floor_digit]} to reach {end}.")
        lines.append(f"Arrived at {end} (Room: {room}) ✅")
    else:
        lines.append(f"From Entrance, reach {end}.")
        lines.append(f"Arrived at {end} ✅")

    # Display directions
    display(Markdown("### 📍 Route Directions"))
    display(Markdown("\n\n".join(lines)))

    # Speak instructions
    speak_steps(lines)

    # Generate map image and display
    img_path = generate_directional_image(G)
    display(Image(img_path))

    # Generate QR code linking to Drive
    generate_qr_for_drive_link(drive_link)

# ----------------- Main -----------------
print("Available destinations:", ", ".join(room_numbers.keys()))
start = "Entrance"
end_input = input("Enter your destination: ").strip()

matches = [name for name in room_numbers if name.lower() == end_input.lower()]

if matches:
    find_route(start, matches[0])
else:
    print("❌ No valid destination provided.")